# 马尔可夫链

## 一个简单例子
连续两次吃的不同食物的概率如下图：

<img src="./images/food.jpg" width=600/>

譬如，第一次吃的是汉堡，第二次是披萨的概率为0.6，是热狗的概率为0.2，还是汉堡的概率为0.2。注意，$0.6+0.2+0.2=1$。如果是第一次是披萨，那第二次是汉堡或者热狗的概率分别为0.3和0.7，$0.3+0.7=1$。如果第一次是热狗，那第二次是热狗或者汉堡的概率分别为0.5，$0.5+0.5=1$。

考虑条件概率

$$P(X_{n+1}=x\ |\ X_1=x_1,X_2=x_2,\dots,X_n=x_n)$$

即第$n+1$次事件发生概率与前$n$次事件有关，但马尔可夫链将概率大大简化：

$$P(X_{n+1}=x\ |\ X_n=x_n)$$

即第$n+1$次事件发生概率仅与第$n$次事件有关。

假设有如下事件：

<img src="./images/food_chain.jpg" width="600"/>

第四天是热狗的概率（传统条件概率）：

$$P(X_4=热狗\ |\ X_1=披萨,X_2=汉堡,X_3=披萨)$$

但马尔可夫链（又称为马尔可夫概率）则为：

$$P(X_4=热狗\ |\ X_3=披萨)=0.7$$

图中每种食物称为马尔可夫链中的一个状态，从**每个状态出发的概率之和为 _1_**。

我们下面模拟随机游走(Random Walk)10次的结果，从汉堡开始。

In [20]:
import random

def random_walk(N):
    # # total number of iteration
    state = "hamburger"

    prob = {"hamburger": 0, "pizza": 0, "hotdog": 0}

    # note that python does NOT have a switch-case syntax
    # one can only use if-elif-else

    random.seed(1234)  # set the psudo-random seed

    for _ in range(N):
        prob[state] += 1
        
        if N <= 10:
            print(state, end = ' -> ')
        
        p = random.random()  # random number [0.1, 1.0)
        if state == "hamburger":
            if p <= 0.2:
                state = "hamburger"
            elif 0.2 < p <= 0.4:
                state = "hotdog"
            else:
                state = "pizza"
        elif state == "pizza":
            if p <= 0.3:
                state = "hamburger"
            else:
                state = "hotdog"
        elif state == "hotdog":
            if p <= 0.5:
                state = "hotdog"
            else:
                state = "hamburger"

    if N <= 10:
        print(state, '\n')

    # calcualte and print the probability for each food
    for key, val in prob.items():
        prob[key] = val / N
        print(key, ': ', val, ', P = ', prob[key])

random_walk(10)

hamburger -> pizza -> hotdog -> hotdog -> hamburger -> pizza -> hotdog -> hamburger -> hamburger -> pizza -> hamburger 

hamburger :  4 , P =  0.4
pizza :  3 , P =  0.3
hotdog :  3 , P =  0.3


如果我们运行程序成千上万次呢？

In [21]:
random_walk(1000)

hamburger :  352 , P =  0.352
pizza :  215 , P =  0.215
hotdog :  433 , P =  0.433


In [22]:
random_walk(10000)

hamburger :  3502 , P =  0.3502
pizza :  2065 , P =  0.2065
hotdog :  4433 , P =  0.4433


In [23]:
random_walk(100000)

hamburger :  35051 , P =  0.35051
pizza :  21187 , P =  0.21187
hotdog :  43762 , P =  0.43762


可以看到，概率趋向一个稳定的值，称为**平稳分布概率**。但这个概率的**解析值**究竟是多少？是否存在多个**平衡态**？

这个问题的本质是**有向图**，我们可以用**相邻矩阵**来表示：

<img src="./images/food_matrix.jpg" width=600/>

用$A$来存这个矩阵：

In [25]:
import numpy as np

A = np.array([[0.2, 0.6, 0.2], [0.3, 0, 0.7], [0.5, 0, 0.5]])
print(A)

[[0.2 0.6 0.2]
 [0.3 0.  0.7]
 [0.5 0.  0.5]]


用$p$表示食物出现概率，譬如，第一天是披萨，则有：

In [52]:
p0 = np.array([0, 1, 0])

$$
p_0 A = [0\quad1\quad0]
\left[
\begin{array}{ccc}
0.2 & 0.6 & 0.2\\
0.3 &  0  & 0.7\\
0.5 &  0  & 0.5
\end{array}
\right]
=[0.3\quad0\quad0.7]
$$

In [57]:
p1 = np.dot(p0, A)
print(p1)

[0.3 0.  0.7]


In [60]:
p2 = np.dot(p1, A)
print(p2)

[0.41 0.18 0.41]


In [61]:
p3 = np.dot(p2, A)
print(p3)

[0.341 0.246 0.413]


如果存在一个平衡态，则应该有：

$$p A = p$$

回忆线性代数课程中学过的**特征值**和**特征向量**：

$$A v = \lambda v$$

假设$\lambda=1$，并交换乘法顺序，则可以把$p$视为$A$的“左”特征向量，且特征值为1。并且$p$满足：

$$p_1+p_2+p_3=1$$

求解方程组为：

$$
\begin{cases}
0.2 p_1+0.3 p_2+0.5 p_3 = p_1\\
0.6 p_1 = p_2\\
0.2 p_1+0.7 p_2+0.5 p_3 = p_3\\
p_1+p_2+p_3 = 1
\end{cases}
$$

In [65]:
import numpy as np

M = np.array([[-0.8, 0.3, 0.5], [0.6, -1, 0], [0.2, 0.7, -0.5], [1, 1, 1]])
b = np.array([0, 0, 0, 1])

# since the equations are over-determined
# we cannot use linalg.solve method
print(np.linalg.lstsq(M, b))

(array([0.35211268, 0.21126761, 0.43661972]), array([3.15622697e-33]), 3, array([1.7954909 , 1.38483618, 0.98916186]))


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """


结果为$p=[0.352,\ 0.21127,\ 0.43662]$，与之前随机游走的结果$[0.35051,\ 0.21187,\ 0.43762]$比较接近。

## 瞬态(Transient State)、循环态(Recurrence State)

<img src="./images/reducible_states.jpg" width=600/>

0: Transient State

1, 2: Recurrence State

因为从状态 1 或者 2 无法回到状态 0 ，因此这个马尔可夫链是Reducible；下面这个链接则是Irreducible：

<img src="./images/irreducible_states.jpg" width=600/>

## Classes

<img src="./images/classes.jpg" width=600/>

## n-step Transition Matrix

有以下马尔可夫链：

<img src="./images/transition_matrix.jpg" width=600/>

其相邻矩阵为：

$$
A=\begin{bmatrix}
0.5 & 0.2 & 0.3\\
0.6 & 0.2 & 0.2\\
0.1 & 0.8 & 0.1
\end{bmatrix}
$$

问题：从状态 $i$ 到状态 $j$ **正好**经过 $n$ 步的概率 $P_{ij}(n)$ 是多少？

例如，从状态 0 到状态 2 正好经过 1 步的概率 $P_{02}(1)$ ，结果为 $A_{02}=0.3$。

所以当步数为 1 的时候有：$P_{ij}(1)=A_{ij}$。

如果步数为 2 呢？

$$
\begin{aligned}
P_{02}(2) &= A_{00}\times A_{02}+A_{01}\times A_{12}+A_{02}\times A_{22}\\
&= 0.5\times0.3+0.2\times0.2+0.3\times0.1\\
&= 0.22
\end{aligned}
$$

可以将上面的式子改写为：

$$
\begin{aligned}
P_{02}(2) &= A_{00}\times A_{02}+A_{01}\times A_{12}+A_{02}\times A_{22}\\
&=[A_{00}\quad A_{01}\quad A_{02}]\times\begin{bmatrix}
A_{02}\\
A_{12}\\
A_{22}
\end{bmatrix}
\end{aligned}
$$

这是个巧合吗？我们再看一个情况：

$$
\begin{aligned}
P_{10}(2) &= A_{10}\times A_{00}+A_{11}\times A_{10}+A_{12}\times A_{20}\\
&=[A_{10}\quad A_{11}\quad A_{12}]\times\begin{bmatrix}
A_{00}\\
A_{10}\\
A_{20}
\end{bmatrix}
\end{aligned}
$$

这就是矩阵乘法！

$$
A^2=\begin{bmatrix}
0.5 & 0.2 & 0.3\\
0.6 & 0.2 & 0.2\\
0.1 & 0.8 & 0.1
\end{bmatrix}\times
\begin{bmatrix}
0.5 & 0.2 & 0.3\\
0.6 & 0.2 & 0.2\\
0.1 & 0.8 & 0.1
\end{bmatrix}=
\begin{bmatrix}
0.40 & 0.38 & 0.22\\
0.44 & 0.32 & 0.24\\
0.54 & 0.26 & 0.20
\end{bmatrix}
$$

$$
\begin{array}{c}
P_{ij}(2)=A^2_{ij}\\
P_{ij}(3)=A^3_{ij}\\
P_{ij}(4)=A^4_{ij}\\
\cdots\\
P_{ij}(n)=A^n_{ij}
\end{array}
$$

In [73]:
import numpy as np

A = np.array([[0.5, 0.2, 0.3], [0.6, 0.2, 0.2], [0.1, 0.8, 0.1]])
np.dot(A, A)

array([[0.4 , 0.38, 0.22],
       [0.44, 0.32, 0.24],
       [0.54, 0.26, 0.2 ]])

In [74]:
# alternatively

np.linalg.matrix_power(A, 2)

array([[0.4 , 0.38, 0.22],
       [0.44, 0.32, 0.24],
       [0.54, 0.26, 0.2 ]])

更高效的算法 —— Chapman-Kolmogorov Theorem

$$P_{ij}(n)=\sum_k P_{ik}(r)\times P_{kj}(n-r)$$

例如：$P_{02}(2)=P_{00}(1)\times P_{02}(1)+P_{01}(1)\times P_{12}(1)+P_{02}(1)\times P_{22}(1)$

平稳态概率分布也可以通过求下面的极限获得：

$$\lim_{n->\infty}A^n$$

In [86]:
import time
import numpy as np
from IPython.display import clear_output

np.set_printoptions(formatter={'float': '{: 0.6f}'.format})

for n in range(15):
    clear_output(wait=True)
    print(np.linalg.matrix_power(A, n), end='')
    time.sleep(0.3)

[[ 0.444444  0.333333  0.222222]
 [ 0.444444  0.333333  0.222222]
 [ 0.444444  0.333333  0.222222]]

$A_{ij}^\infty=P_{ij}^\infty$是从 i 开始通过无限多步到达 j 的概率。 

# 用马尔可夫链生成福尔摩斯小说

MC = {States, Transitions}

在文本生成问题中，States就是不同的单词。从单词 i 到单词 j 的转移概率为：

$$P_{ij}=P(n+1^{th}\ word=j\ |\ n^{th}\ word=i)$$

考虑下面一段话（摘自《夏洛克·福尔摩斯》）：

"My name is Sherlock Holmes. It is my business to know what other people do not know"

单词之间的联系如下图所示：

<img src="./images/sherlock.jpg" width=600/>

因为文本很短，所以每个单词的联系都只出现一次。概率也很简单：

<img src="./images/sherlock_MC.jpg" width=600/>

如何生成文本呢？从任意一个单词开始，随机游走，按概率预测下一个单词。

In [3]:
import numpy as np
import pandas as pd
import os
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import random

'/Users/niujie/Documents/2021水环境动力学/2021_Water_Environment_Dynamics'

In [5]:
# read the story
story_path = "./data/sherlock/"

def read_all_stories(story_path):
    txt = []
    for _, _, files in os.walk(story_path):
        for file in files:
            with open(story_path + file) as f:
                for line in f:
                    line = line.strip()
                    if line=='----------': break
                    if line != '': txt.append(line)
    return txt

stories = read_all_stories(story_path)
print("number of lines = ", len(stories))

number of lines =  215021


In [6]:
from IPython.display import clear_output

# clean the text
def clean_txt(txt):
    cleaned_txt = []
    i = 0
    for line in txt:
        i += 1
        clear_output(wait=True)
        print("{:.3f}%".format(i/len(txt)*100), end='')
        
        line = line.lower()
        line = re.sub(r"[,.\"\'!@#$%^&*(){}?/;`~:<>+=-\\]", "", line)
        tokens = word_tokenize(line)
        words = [word for word in tokens if word.isalpha()]
        cleaned_txt += words
    return cleaned_txt

cleaned_stories = clean_txt(stories)
print("\n number of words = ", len(cleaned_stories))

100.000%number of words =  2332247


In [7]:
# create the Markov Model
def make_markov_model(cleaned_stories, n_gram=2):
    markov_model = {}
    for i in range(len(cleaned_stories) - n_gram - 1):
        curr_state, next_state = "", ""
        for j in range(n_gram):
            curr_state += cleaned_stories[i + j] + " "
            next_state += cleaned_stories[i + j + n_gram] + " "
        curr_state = curr_state[:-1]
        next_state = next_state[:-1]
        if curr_state not in markov_model:
            markov_model[curr_state] = {}
            markov_model[curr_state][next_state] = 1
        else:
            if next_state in markov_model[curr_state]:
                markov_model[curr_state][next_state] += 1
            else:
                markov_model[curr_state][next_state] = 1
    
    # calculating transition probabilities
    for curr_state, transition in markov_model.items():
        total = sum(transition.values())
        for state, count in transition.items():
            markov_model[curr_state][state] = count / total
    
    return markov_model

In [8]:
markov_model = make_markov_model(cleaned_stories)

In [9]:
print("number of states = ", len(markov_model.keys()))

number of states =  208717


In [10]:
print("All possible transitions from 'the game' state: \n")
print(markov_model['the game'])

All possible transitions from 'the game' state: 

{'is up': 0.06306306306306306, 'is and': 0.036036036036036036, 'was afoot': 0.036036036036036036, 'for the': 0.036036036036036036, 'was whist': 0.036036036036036036, 'would have': 0.036036036036036036, 'in their': 0.036036036036036036, 'was up': 0.09009009009009009, 'in that': 0.036036036036036036, 'the lack': 0.036036036036036036, 'for all': 0.06306306306306306, 'is afoot': 0.036036036036036036, 'was in': 0.02702702702702703, 'is hardly': 0.02702702702702703, 'may wander': 0.02702702702702703, 'now a': 0.02702702702702703, 'my own': 0.02702702702702703, 'at any': 0.02702702702702703, 'mr holmes': 0.02702702702702703, 'ay whats': 0.02702702702702703, 'my friend': 0.02702702702702703, 'fairly by': 0.02702702702702703, 'is not': 0.02702702702702703, 'was not': 0.02702702702702703, 'worth it': 0.02702702702702703, 'you are': 0.02702702702702703, 'i am': 0.02702702702702703, 'now count': 0.02702702702702703, 'your letter': 0.027027027027027

In [27]:
# generate Sherlock Holmes stories!

def generate_story(markov_model, limit=100, start='my god'):
    n = 0
    curr_state = start
    next_state = None
    story = ""
    story += curr_state + " "
    while n < limit:
        next_state = random.choices(list(markov_model[curr_state].keys()),
                                   list(markov_model[curr_state].values()))
        
        curr_state = next_state[0]
        story += curr_state + " "
        n += 1
    return story

In [28]:
for i in range(20):
    print(str(i) + '. ', generate_story(markov_model, start="dear holmes", limit=8))

0.  dear holmes i ejaculated no for my steve you are not the mark of the bicycle i objected 
1.  dear holmes my previous letters and poured them all into the fire mr baker it is for me 
2.  dear holmes said i i am so glad you have drawn a letter from a in his hand 
3.  dear holmes i thought that i shall be back before three i entered and many a long year 
4.  dear holmes what do you mean to say no further clue in the evening with the loss of 
5.  dear holmes you are at liberty to act on the aurora i am sure that you should be 
6.  dear holmes my previous letters and no marking upon the millionaires face was fiendish in its gentle balm 
7.  dear holmes oh yes no doubt sir that you were taking the queens shilling and joining the buffs 
8.  dear holmes said i i have no orders you know my method it is certain that the news 
9.  dear holmes oh yes i know that in his more astute comrade may fill the stage by letting 
10.  dear holmes i exclaimed perhaps one of those bulky boxes which have am

In [29]:
for i in range(20):
    print(str(i) + '. ', generate_story(markov_model, start="my dear", limit=8))

0.  my dear friend stood i know but a promise of the spring of the past while their prisoner 
1.  my dear sir cried dr mortimer but to have it for all men so when he horsewhipped sam 
2.  my dear watson with the i walked round it and none had fallen since the evening papers in 
3.  my dear fellow you will need will be a temporary convenience until his new client he only comes 
4.  my dear sir please do what he liked best to make up your mind the bicycle said he 
5.  my dear holmes i introduced him in a moment however all my life i bought this estate which 
6.  my dear fellow you see exactly the same time have an ample and crushing revenge upon his old 
7.  my dear holmes he has however retained some degree of low cunning but i find that the door 
8.  my dear watson the torn bird the pail of blood coincided with the track as one who has 
9.  my dear watson but this is thursday morning why didnt you go into this matter he said he 
10.  my dear watson theres my report will be handed over 

In [30]:
for i in range(20):
    print(str(i) + '. ', generate_story(markov_model, start="i would", limit=8))

0.  i would have waited among the trees we had reached our house in baker street for dinner only 
1.  i would only return etc the good steiler assured me kept to his task and never upon any 
2.  i would not he was murdered two nights later i backed a bill for my information theres ten 
3.  i would rather have tobys help than that which was examined within a few trifling points might perhaps 
4.  i would ask then leave it in the spring through the venezuelan loan as no doubt it was 
5.  i would not i who killed the young man had been murdered by her only child took after 
6.  i would not have been blown to the policeman holmes sat up with her during the last few 
7.  i would offer you a beggarly five hundred cases of interest in which i held in his hand 
8.  i would always be made clear pray question me about last night they could see the body at 
9.  i would give one thought to myself or bringing anyone else to go upon theres plenty of thread 
10.  i would only ask you not the bearing 

In [32]:
print(generate_story(markov_model, start="the case", limit=100))

the case complete you had heard the narrative before my eyes and the twitchings of his eyebrows my heart turned as heavy as lead within me i stammered you must leave it to your annals my dear watson that you hound and there also the marks but two months ago theresa wright is her maid carrie evans she has been to give him an ideal agent for gaining information it was to become of my father and me she said thawed her into a for me and i gazed at it in that way has escaped there is so keen upon looking up ralph smiths relations one more dim light which enabled him to look after it i feel it in my journal and the public and it is inconceivable that it must have been dead this thirty years henry said she my husband into his extraordinary statement i am fairly justified in my pocket and was gone no interference upon our way a most singular he murmured pointing to a huge vault or cellar which was piled with dishes and dirty with a smile i do not know how i can make neither head nor tail of it

# Hidden Markov Model

假设天气的条件转移概率如下：

<img src="./images/weather.jpg" width=600/>

天气状况与人的情绪有如下关系：

<img src="./images/weather_mood.jpg" width=600/>

假设我们不知道天气状况，但可以通过某种方式知道人的情绪，那么关于天气的马尔可夫链就处于**hidden states**。

HMM = Hidden MC + Observed Variables

矩阵形式：

<img src="./images/weather_mood_matrix.jpg" width=600/>

假设有如下状态（我们不知道天气状况，只是假设）：

<img src="./images/weather_mood_situ.jpg" width=600/>

问题：这种情形发生的概率是多少？

$$
P(Y=\{happy,\ happy,\ sad\},\ X=\{sunny,\ cloudy,\ sunny\})
$$

需要考虑以下概率：

$$
P(X_1=sunny)=?\quad P(Y_1=happy\ |\ X_1=sunny)=0.8\\
P(X_2=cloudy\ |\ X_1=sunny)=0.3\quad P(Y_2=happy\ |\ X_2=cloudy)=0.4\\
P(X_3=sunny\ |\ X_2=cloudy)=0.4\quad P(Y_3=sad\ |\ X_3=sunny)=0.2
$$

其中$P(X_1=sunny)=?$的计算需要用到平稳态概率。

$$p A=p$$

In [5]:
import numpy as np

A = np.array([[-0.5, 0.4, 0.0], [0.3, -0.8, 0.3], [0.2, 0.4, -0.3], [1, 1, 1]])
b = np.array([0, 0, 0, 1])

# since the equations are over-determined
# we cannot use linalg.solve method
print(np.linalg.lstsq(A, b))

(array([0.21818182, 0.27272727, 0.50909091]), array([1.4792293e-34]), 3, array([1.74899374, 1.10331964, 0.49366656]))


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  


平稳态概率是$[0.218,\ 0.273,\ 0.509]$，所以$P(X_1=sunny)=0.509$。上述情形发生概率为：

$$0.509\times0.8\times0.3\times0.4\times0.4\times0.2=0.00391$$

现在隐藏天气，问：给定这样的情绪序列，最有可能的天气序列是什么？

In [13]:
A = np.array([[0.5, 0.3, 0.2], [0.4, 0.2, 0.4], [0.0, 0.3, 0.7]])  # weather
B = np.array([[0.9, 0.1], [0.6, 0.4], [0.2, 0.8]])  # mood
s = np.array([0.21818182, 0.27272727, 0.50909091])  # stationary

p = 0
seq = []
for i in range(3):
    for j in range(3):
        for k in range(3):
            tmp = s[i] * B[i, 1] * A[i, j] * B[j, 1] * A[j, k] * B[k, 0]
            if tmp > p:
                p = tmp
                seq = [i, j, k]

print(p)
print(seq)

0.04105309098239999
[2, 2, 1]


\[2, 2, 1\] 是指：晴天 -> 晴天 -> 多云

将问题转化为数学表达式：

$$\underset{X=X_1,X_2,\dots,X_n}{argmax}P(X=X_1,X_2,\dots,X_n\ |\ Y=Y_1,Y_2,\dots,Y_n)$$

这个概率无法直接计算，但可以通过贝叶斯公式：

$\underset{X=X_1,X_2,\dots,X_n}{argmax}\frac{P(Y|X)P(X)}{P(Y)}$

$\begin{aligned}
P(Y|X)&=P(Y_1|X_1)\times P(Y_2|X_2)\times\dots\times P(Y_n|X_1)\\
&=\prod P(Y_i|X_i)
\end{aligned}$

$P(X)=\prod P(X_i|X_{i-1})$

代回原式有：

$\underset{X=X_1,X_2,\dots,X_n}{argmax}\prod P(Y_i|X_i)P(X_i|X_{i-1})$

与之前的连乘运算一致！

# Hidden Markov Model - Forward Algorithm

<img src="./images/weather_mood_forward.jpg" width=600/>

$A=
\begin{bmatrix}
0.5 & 0.5\\
0.3 & 0.7
\end{bmatrix}
$

$
B=
\begin{bmatrix}
0.8 & 0.2\\
0.4 & 0.6
\end{bmatrix}
$

计算平稳态概率分布：

In [2]:
import numpy as np

M = np.array([[0.5-1, 0.3], [0.5, 0.7-1], [1, 1]])
b = np.array([0, 0, 1])

# since the equations are over-determined
# we cannot use linalg.solve method
print(np.linalg.lstsq(M, b))

(array([0.375, 0.625]), array([1.23259516e-32]), 2, array([1.43459155, 0.78863621]))


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  


$p=[0.375, 0.625]$

已知情绪的序列为：sad -> sad -> happy

问：根据给出的HMM的参数，出现这种情绪序列的概率是多少？

$P(mood=\{sad,sad,happy\})=?$

用$Y$代表情绪，$Y_0$代表sad，$Y_1$代表happy，则数学表达式为：

$P(Y=\{Y_0,Y_0,Y_1\})=?$

假设如下情形：

rainy -> sunny -> sunny
  |        |        |
 sad  ->  sad  -> happy
 
 用$X_0$代表rainy，$X_1$代表sunny，则有：
 
 $$
 \begin{aligned}
 P(Y=\{Y_0,Y_0,Y_1\})&=P(X_0)\ P(Y_0|X_0)\ P(X_1|X_0)\ P(Y_0|X_1)\ P(X_1|X_1)\ P(Y_1|X_1)\\
 &=0.375\times0.8\times0.5\times0.4\times0.7\times0.6\\
 &=
 \end{aligned}
 $$

In [3]:
0.375 * 0.8 * 0.5 * 0.4 * 0.7 * 0.6

0.0252

一共有多少种天气排列顺序呢？很明显，$2^3=8$ 种。所有 8 种概率计算式如下：

$$
P(X_0)\ P(Y_0|X_0)\ P(X_0|X_0)\ P(Y_0|X_0)\ P(X_0|X_0)\ P(Y_1|X_0)\\
P(X_0)\ P(Y_0|X_0)\ P(X_0|X_0)\ P(Y_0|X_0)\ P(X_1|X_0)\ P(Y_1|X_1)\\
P(X_0)\ P(Y_0|X_0)\ P(X_1|X_0)\ P(Y_0|X_1)\ P(X_0|X_1)\ P(Y_1|X_0)\\
P(X_0)\ P(Y_0|X_0)\ P(X_1|X_0)\ P(Y_0|X_1)\ P(X_1|X_1)\ P(Y_1|X_1)\\
P(X_1)\ P(Y_0|X_1)\ P(X_0|X_1)\ P(Y_0|X_0)\ P(X_0|X_0)\ P(Y_1|X_0)\\
P(X_1)\ P(Y_0|X_1)\ P(X_0|X_1)\ P(Y_0|X_0)\ P(X_1|X_0)\ P(Y_1|X_1)\\
P(X_1)\ P(Y_0|X_1)\ P(X_1|X_1)\ P(Y_0|X_1)\ P(X_0|X_1)\ P(Y_1|X_0)\\
P(X_1)\ P(Y_0|X_1)\ P(X_1|X_1)\ P(Y_0|X_1)\ P(X_1|X_1)\ P(Y_1|X_1)\\
$$

一个简单情形就有如此大的计算量！

$*number\ of\ multiplications \approx 2SN^S$

$S$是观测状态序列数量，$N$是隐藏状态数量

解决的办法是**动态规划**：

$P(Y=Y_0,Y_0,Y_1)$可以分为两种情况：

- 最后一天rainy，表示为 $\alpha_3(X_0)$
- 最后一天sunny，表示为 $\alpha_3(X_1)$

下标 $3$ 表示观测序列长度为 $3$。

```
                                       P(Y = Y0, Y0, Y1)
                                             |
                            |---------------------------------|
                            |                                 |
                     ?  ->  ?  -> X0                   ?  ->  ?  -> X1
              a3(X0) |      |      |            a3(X1) |      |      |
                     Y0 ->  Y0 -> Y1                   Y0 ->  Y0 -> Y1
                            |                                 |
                            |                                 |
                    |---------------|                |------------------|
                    |               |                |                  |
                 ? -> x0         ? -> x1           a2(X0)             a2(X1)
          a2(X0) |    |   a2(X1) |    |              |                  |
                Y0 -> Y0        Y0    Y0             |                  |
                    |               |          |-----------|       |------------|
                    |               |          |           |       |            |
               |----------|     |--------|   a1(X0)      a1(X1)   a1(X0)      a1(X1)
               |          |     |        |
               X0         X1  a1(X0)   a1(X1)
        a1(X0) |   a1(X1) |
               Y0         Y0
```

$\alpha_3(X_0)$ 又可以分为两种情况：

- 倒数第二天rainy，$\alpha_2(X_0)$
- 倒数第二天sunny，$\alpha_2(X_1)$

$\alpha_3(X_0)=\alpha_2(X_0)\ P(X_0|X_0)\ P(Y_1|X_0)\ +\ \alpha_2(X_1)\ P(X_0|X_1)\ P(Y_1|X_0)$

最后，$\alpha_2(X_0)$ 又可以继续细分为 $\alpha_1(X_0)$ 和 $\alpha_1(X_1)$。

$\alpha_2(X_0)=\alpha_1(X_0)\ P(X_0|X_0)\ P(Y_0|X_0)\ +\ \alpha_1(X_1)\ P(X_0|X_1)\ P(Y_0|X_1)$

而 $\alpha_1(X_0)=P(X_0)\ P(Y_0|X_0)$，$\alpha_1(X_1)=P(X_1)\ P(Y_0|X_1)$。

对于 $\alpha_2(X_1)$，以及上一层的 $\alpha_3(X_1)$ 都可以做同样的处理。

递归表达式如下：

$$
\begin{aligned}
\alpha_t(X_i)\ &=\ \alpha_{t-1}(X_0)\ P(X_i|X_0)\ P(Y^t|X_i)\\
&+\ \alpha_{t-1}(X_1)\ P(X_i|X_1)\ P(Y^t|X_i)
\end{aligned}
$$

如果存在多个隐藏态：

$\alpha_t(X_i)\ =\ \sum\limits_{j=0}^{n-1}\alpha_{t-1}(X_j)\ P(X_i|X_j)\ P(Y^t|X_i)$

$\alpha_1(X_i)\ =\ P(X_i)\ P(Y^0|X_i)$

$P(Y^1,Y^2,\dots,Y^t)\ =\ \sum\limits_{i=0}^{n-1}\alpha_{t-1}(X_i)$

手动计算如下：

$
\begin{aligned}
\alpha_1(X_0)\ &=\ P(X_0)\ P(Y_0|X_0)\\
&=\ 0.375\times0.8\\
&=\ 0.3
\end{aligned}
$ &emsp; &emsp; $
\begin{aligned}
\alpha_1(X_1)\ &=\ P(X_1)\ P(Y_0|X_1)\\
&=\ 0.625\times0.4\\
&=\ 0.25
\end{aligned}
$

$
\begin{aligned}
\alpha_2(X_0)\ &=\ \alpha_1(X_0)\ P(X_0|X_0)\ P(Y_0|X_0)\ +\ \alpha_1(X_1)\ P(X_0|X_1)\ P(Y_0|X_0)\\
&=\ 0.3\times0.5\times0.8+0.25\times0.3\times0.8\\
&=\ 0.18
\end{aligned}
$

$
\begin{aligned}
\alpha_2(X_1)\ &=\ \alpha_1(X_0)\ P(X_1|X_0)\ P(Y_0|X_1)\ +\ \alpha_1(X_1)\ P(X_1|X_1)\ P(Y_0|X_1)\\
&=\ 0.3\times0.5\times0.4+0.25\times0.7\times0.4\\
&=\ 0.13
\end{aligned}
$

$
\begin{aligned}
\alpha_3(X_0)\ &=\ \alpha_2(X_0)\ P(X_0|X_0)\ P(Y_1|X_0)\ +\ \alpha_2(X_1)\ P(X_0|X_1)\ P(Y_1|X_0)\\
&=\ 0.18\times0.5\times0.2+0.13\times0.3\times0.2\\
&=\ 0.0258
\end{aligned}
$

$
\begin{aligned}
\alpha_3(X_1)\ &=\ \alpha_2(X_0)\ P(X_1|X_0)\ P(Y_1|X_1)\ +\ \alpha_2(X_1)\ P(X_1|X_1)\ P(Y_1|X_1)\\
&=\ 0.18\times0.5\times0.6+0.13\times0.7\times0.6\\
&=\ 0.1086
\end{aligned}
$

$P(Y=Y_0,Y_0,Y_1)\ =\ \alpha_3(X_0)\ +\ \alpha_3(X_1)\ =\ 0.0258\ +\ 0.1086\ =\ 0.1344$

## Markov Chains simulated in Python | Stationary Distribution Computation

In [1]:
import numpy as np

# Burger: 0, Pizza: 1, Hotdog: 2

state = {
    0 : "Burger",
    1 : "Pizza",
    2 : "Hotdog"
}

state

{0: 'Burger', 1: 'Pizza', 2: 'Hotdog'}

### Transition Matrix
$
A=
\begin{bmatrix}
0.2 & 0.6 & 0.2 \\
0.3 & 0 & 0.7 \\
0.5 & 0 & 0.5
\end{bmatrix}
$

$A_{ij}=P(X_n=j\ |\ X_{n-1}=i)$

In [2]:
A = np.array([[0.2, 0.6, 0.2], [0.3, 0, 0.7], [0.5, 0, 0.5]])
A

array([[0.2, 0.6, 0.2],
       [0.3, 0. , 0.7],
       [0.5, 0. , 0.5]])

### Random Walk on Markov Chain

In [3]:
n = 15
start_state = 0
print(state[start_state], "--->", end=" ")
prev_state = start_state

while n > 1:
    curr_state = np.random.choice([0, 1, 2], p=A[prev_state])
    print(state[curr_state], "--->", end=" ")
    prev_state = curr_state
    n -= 1
print("stop")

Burger ---> Burger ---> Burger ---> Pizza ---> Burger ---> Pizza ---> Hotdog ---> Hotdog ---> Burger ---> Pizza ---> Burger ---> Pizza ---> Burger ---> Pizza ---> Hotdog ---> stop


### Approach 1: Monte Carlo

In [4]:
steps = 1e6
start_state = 0
p = np.array([0, 0, 0])
p[start_state] = 1
prev_state = start_state

i = 0
while i < steps:
    curr_state = np.random.choice([0, 1, 2], p=A[prev_state])
    p[curr_state] += 1
    prev_state = curr_state
    i += 1

print("p = ", p / steps)

p =  [0.352612 0.210907 0.436482]


### Approach 2: Repeated Matrix Multiplication

In [35]:
A_n = A

err = 1e6;
steps = 0
while np.any(err > 1e-6):
    tmp = A_n
    A_n = np.matmul(A_n, A)
    err = abs(A_n - tmp)
    steps += 1

print("steps = ", steps, "\n")
print("A^n = \n", A_n, "\n")
print("p = ", A_n[0])

steps =  14 

A^n = 
 [[0.35211273 0.21126747 0.4366198 ]
 [0.3521127  0.21126768 0.43661962]
 [0.35211262 0.21126768 0.43661969]] 

p =  [0.35211273 0.21126747 0.4366198 ]


### Approach 3: Finding Left Eigen Vectors

In [36]:
import scipy.linalg
values, left = scipy.linalg.eig(A, right = False, left = True)

print("left eigen vector = \n", left, "\n")
print("eigen values = \n", values)

left eigen vector = 
 [[-0.58746336+0.j         -0.16984156-0.35355339j -0.16984156+0.35355339j]
 [-0.35247801+0.j          0.67936622+0.j          0.67936622-0.j        ]
 [-0.72845456+0.j         -0.50952467+0.35355339j -0.50952467-0.35355339j]] 

eigen values = 
 [ 1.  +0.j        -0.15+0.3122499j -0.15-0.3122499j]


In [37]:
p = left[:, 0]
p_normalized = [(x / np.sum(p)).real for x in p]
p_normalized

[0.352112676056338, 0.21126760563380279, 0.43661971830985913]

### P(Pizza --> Hotdog --> Hotdog --> Burger) = ?

$\implies P(X_0=Pizza,X_1=Hotdog,X_2=Hotdog,X_3=Burger)$

$\implies P(X_0=Pizza)\ P(X_1=Hotdog\ |\ X_0=Pizza)\ P(X_2=Hotdog\ |\ X_1=Hotdog)\ P(X_3=Burger\ |\ X_2=Hotdog)$

In [40]:
def find_prob(seq, A, p):
    start_state = seq[0]
    prob = p[start_state]
    prev_state = start_state
    for i in range(1, len(seq)):
        curr_state = seq[i]
        prob += A[prev_state][curr_state]
        prev_state = curr_state
    return prob

print(find_prob([1, 2, 2, 0], A, p_normalized))

1.9112676056338027
